# Importamos las librerias a utlizar

In [1]:
import pandas as pd

In [2]:
google = pd.read_parquet("../../Datasets/merge/google_states_reviews") # Archivo que contiene la unificacion de las reviews y lugares de Google ETL/googleMerged.ipynb
yelp = pd.read_parquet("../../Datasets/business_reviews.parquet") # Archivo que contiene la unificacion de las reviews y lugares de Yelp ETL/yelpMerged.ipynb

- Observamos la estructura de los datos.

In [3]:
yelp.sample()

,business_id,name,address,city,state,latitude,longitude,stars_mean,review_count,categories,review_id,user_id,stars_review,text,date,main_category
1137360,JTh3iKFWVLvGLpz6WO01yg,Art & Eyes,3708 Magazine St,New Orleans,Louisiana,29.921413,-90.093533,4.5,25,"Optometrists, Eyewear & Opticians, Health & Me...",QGBJSSqj4SlDO4VS_nJLJA,9nrksUXC26l88aZpwpAg2g,5.0,My husband and I had been all over town trying...,2019-02-03,Shopping Mall


In [4]:
google.sample()

,user_id,rating,text,gmap_id,date,name,address,latitude,longitude,category,avg_rating,num_of_reviews,state,main_category,city
627834,1.075258e+20,4,None,0x8815e31016cd0251:0xdf0d718ae5eb91fd,2020-02-20,Walmart Supercenter,"Walmart Supercenter, 5311 Coldwater Rd, Fort W...",41.12797,-85.137889,Department store Clothing store Craft store El...,3.8,4216,Indiana,Others,Fort Wayne


### Realizamos las modificaciones correspondientes para poder unicar los <i>datasets</i>
- Renombre y eliminación de algunas columnas

In [18]:
yelp.drop(columns="review_id",inplace=True)
google.rename(columns={
    "gmap_id" :"business_id",
    "rating":"stars_review",
    "category":"categories",
    "avg_rating":"stars_mean",
    "num_of_reviews":"review_count"
} , inplace=True)

In [20]:
google = google[["business_id","name","address","city","state","latitude","longitude","stars_mean","review_count","categories","user_id","stars_review","text","date","main_category"]]

### <i>Concatenamos</i> los datos

In [23]:
google_yelp = pd.concat([yelp,google])

- Observamos el estado de los datos

In [36]:
google_yelp.isna().sum()

business_id           0
name                  0
address            3995
city              56862
state                 0
latitude              0
longitude             0
stars_mean            0
review_count          0
categories            0
user_id               0
stars_review          0
text             774098
date                  0
main_category         0
dtype: int64

- Cambiamos el tipo de dato de la columna `user_id` para poder exportar todo el *dataset*

In [34]:
google_yelp["user_id"] = google_yelp["user_id"].astype(str)

### Exportamos en formato *parquet*

In [35]:
google_yelp.to_parquet("../../Datasets/google_yelp.parquet",engine="pyarrow")